## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.eefolium`](https://github.com/giswqs/geemap/blob/master/geemap/eefolium.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/giswqs/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [ ]:
!pip install geemap


In [ ]:
# Installs geemap package
import google.colab
import geemap.eefolium as geemap

# Authenticates and initializes Earth Engine
import ee

ee.Authenticate()
ee.Initialize()  

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=eCMJdGktuXY7K4Se1-oTy5NJIKMSE2HYjY44rArEhZU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWi7oT1_tBE205cU0qIohXL6lYUoX1B_M-bfzFaUa4ws3foO7D_iiaM

Successfully saved authorization token.


## Create an interactive map 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [ ]:
Map = geemap.Map(center=[32.3,-90.9], zoom= 11)
Map

## Add Earth Engine Python script 

In [ ]:
# Import the Sentinel 2 collection as sentinel 2
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

#Coordinates for the bounds of a rectangle.
xMin = -91.2
yMin = 32
xMax = -90.8
yMax = 32.7

#Construct a rectangle from a list of GeoJSON 'point' formatted coordinates.
rectangleGeoJSON = ee.Geometry.Rectangle(
  [
    [xMin, yMin],
    [xMax, yMax]   # max x and y
  ]
)

miss_bnds = ee.FeatureCollection([
    ee.Feature(
        rectangleGeoJSON,
        {'name': 'Mississippi Vicksburg', 'fill': 1}),
    ])

# Filter the image collection using filterBounds() and filterDate() method.
# Sort the collection by cloud cover metadata
# Create the mosaic image and clip it to Cambodia boundary
miss_img_pre = sentinel2.filterBounds(miss_bnds).filterDate('2019-9-01', '2019-10-01').mosaic().clip(miss_bnds)

miss_img_post = sentinel2.filterBounds(miss_bnds).filterDate('2019-03-15', '2019-05-01').mosaic().clip(miss_bnds)

# Calculate NDWI                  
#ndwi_dry = image_dry.normalizedDifference(['B3', 'B8']);
#ndwi_wet = image_wet.normalizedDifference(['B3', 'B8']);

# Palettes for Visualization
trueColor_palette = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000};
waterPalette = ['red', 'yellow', 'green', 'blue'];
#print(miss_bnds)


## Display Earth Engine True Color Images layers in Dry Season

In [ ]:
Map = geemap.Map(center=[32.3,-90.9], zoom= 12)
Map.addLayer(miss_img_pre, trueColor_palette, '2019 Dry season true color');
Map.addLayerControl()
Map

## Display Earth Engine True Color Images layers in Wet Season

In [ ]:
Map = geemap.Map(center=[32.3,-90.9], zoom= 12)
Map.addLayer(miss_img_post, trueColor_palette, '2019 Wet season true color');
Map.addLayerControl()
Map

## Display Earth Engine the NDWI images with a color palette in Dry Season

In [ ]:
Map = geemap.Map(center=[12,105], zoom= 8)
Map.addLayer(ndwi_dry, {'min': -1, 'max': 0.5, 'palette': waterPalette}, '2019 Dry season NDWI');
Map

NameError: ignored

## Display Earth Engine the NDWI images with a color palette in Wet Season

In [ ]:
Map = geemap.Map(center=[12,105], zoom= 8)
# Display NDWI Wet
Map.addLayer(ndwi_wet, {'min': -1, 'max': 0.5, 'palette': waterPalette}, '2019 Wet season NDWI');
Map